In [1]:
import os
import librosa
import Loader
import numpy as np
import librosa.display
import tensorflow as tf
import matplotlib.pyplot as plt

In [2]:
def Trim(Signal,Sr,Len = 5):
    Signal = np.array(Signal[:(Len*Sr)])
    return Signal
    

def Pad(Signal,Len,Sr = 22050):
    if len(Signal) < (Len*Sr):
        Pad_Size = (Len*Sr)-len(Signal)
        Padded = np.pad(Signal, (0,Pad_Size ), 'constant')
        return Padded
    return Signal

def Waveform(Signal,Sr):
    librosa.display.waveplot(Signal,sr=Sr)
    plt.xlabel("Time")
    plt.ylabel("Amplitude")
    plt.show()


def FFT(Signal,Sr):
    fft = np.fft.fft(Signal)
    magnitude = np.abs(fft)
    freq = np.linspace(0,Sr,len(magnitude))
    freq = freq[:int(len(freq)/2)]
    magnitude = magnitude[:int(len(magnitude)/2)]
    plt.plot(freq,magnitude)
    plt.xlabel("Frequency")
    plt.ylabel("Magnitude")
    plt.show()



def Sepctogram(Signal,Sr,Mode = 'Normal'): #Using STFT
    Sample_Num = 2048 #Per fft
    Hop_length = 512 
    stft = librosa.core.stft(Signal,hop_length=Hop_length,n_fft=Sample_Num)
    spect = np.abs(stft)
    if Mode == 'log':
        spect = librosa.amplitude_to_db(spect)
    librosa.display.specshow(spect,sr=Sr,hop_length = Hop_length)
    plt.xlabel("Time")
    plt.ylabel("Frequency")
    plt.colorbar()
    plt.show()

def MFCC(Signal,Sr):
    Sample_Num = 2048 #Per fft
    Hop_length = 512 
    MFCC = librosa.feature.mfcc(Signal,sr=Sr,n_fft=Sample_Num,hop_length = Hop_length,n_mfcc=20)
    librosa.display.specshow(MFCC,sr=Sr,hop_length = Hop_length)
    plt.xlabel("Time")
    plt.ylabel("MFCC")
    plt.colorbar()
    plt.show()

def Save_MFCC(Signal,Sr,n_mfcc=13,n_fft=2048,hop_length=512):
    MFCC = librosa.feature.mfcc(Signal,sr=Sr,n_fft=n_fft,hop_length = hop_length,n_mfcc=n_mfcc)
    return MFCC/303.0

In [4]:
Pain_Path,Hungry_Path,Uncomfortable_Path = Loader.LOAD()
Audio_Hungry = []
Audio_Pain = []
Audio_UnCom = []
for path in Hungry_Path:
    Signal,Sr = librosa.load(path,sr=22050) #len(Signal) = sr * AudioTime  
    Audio_Hungry.append(Trim(Pad(Signal,5,Sr),Sr))

for path in Pain_Path:
    Signal,Sr = librosa.load(path,sr=22050) #len(Signal) = sr * AudioTime  
    Audio_Pain.append(Trim(Pad(Signal,5,Sr),Sr))

for path in Uncomfortable_Path:
    Signal,Sr = librosa.load(path,sr=22050) #len(Signal) = sr * AudioTime  
    Audio_UnCom.append(Trim(Pad(Signal,5,Sr),Sr))

print('Audio Loaded')

Audio Loaded


In [5]:
np.random.shuffle(Audio_Pain)
np.random.shuffle(Audio_UnCom)
np.random.shuffle(Audio_Hungry)

In [11]:
Data = []
Y = []

for i in range(len(Audio_UnCom) + 100):
    Data.append(Save_MFCC(Audio_Hungry[i],Sr))
    Y.append(0)

for Audio_File in Audio_Pain:
    Data.append(Save_MFCC(Audio_File,Sr))
    Y.append(1)

for Audio_File in Audio_UnCom:
    Data.append(Save_MFCC(Audio_File,Sr))
    Y.append(2)

Data = np.array(Data)
Y = np.array(Y)


c:\Users\Admin\miniconda3\envs\TF-2.5\lib\site-packages\ipykernel_launcher.py:57: FutureWarning: Pass y=[ 3.8349244e-06 -4.4584217e-06  5.1752477e-06 ...  0.0000000e+00
  0.0000000e+00  0.0000000e+00] as keyword args. From version 0.10 passing these as positional arguments will result in an error
c:\Users\Admin\miniconda3\envs\TF-2.5\lib\site-packages\ipykernel_launcher.py:57: FutureWarning: Pass y=[-1.5049471e-03 -1.0572501e-03 -4.4646284e-05 ...  1.4461914e-01
  1.7091589e-01  2.1611854e-01] as keyword args. From version 0.10 passing these as positional arguments will result in an error
c:\Users\Admin\miniconda3\envs\TF-2.5\lib\site-packages\ipykernel_launcher.py:57: FutureWarning: Pass y=[-4.2985441e-05 -2.7570080e-05  2.0100257e-05 ...  4.2655516e-02
  6.6667996e-02  7.0479214e-02] as keyword args. From version 0.10 passing these as positional arguments will result in an error
c:\Users\Admin\miniconda3\envs\TF-2.5\lib\site-packages\ipykernel_launcher.py:57: FutureWarning: Pass y=[-

In [7]:
Split = int(len(Data)*0.90)
X_Train = Data[:Split]
Y_Train = Y[:Split]
X_Test = Data[Split:]
Y_Test = Y[Split:]

In [8]:
Model = tf.keras.Sequential([
        tf.keras.layers.Conv1D(20,5,activation='relu',padding='same',input_shape = (13,216)),
        tf.keras.layers.MaxPool1D(2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512,activation= 'relu'),
    tf.keras.layers.Dense(256,activation= tf.keras.activations.elu),
    tf.keras.layers.Dense(128,activation='relu'),   
    tf.keras.layers.Dense(3,activation='softmax')
])
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
Model.compile(optimizer=optimizer,loss="sparse_categorical_crossentropy",metrics = ["accuracy"])
Hist = Model.fit(X_Train,Y_Train,epochs=25,batch_size=6,validation_data=(X_Test,Y_Test))


Epoch 1/25
83/83 [==============================] - 1s 7ms/step - loss: 0.9424 - accuracy: 0.5903 - val_loss: 1.5034 - val_accuracy: 0.0182
Epoch 2/25
83/83 [==============================] - 0s 4ms/step - loss: 0.8853 - accuracy: 0.6288 - val_loss: 1.4708 - val_accuracy: 0.3636
Epoch 3/25
83/83 [==============================] - 0s 4ms/step - loss: 0.7864 - accuracy: 0.6897 - val_loss: 1.3272 - val_accuracy: 0.4000
Epoch 4/25
83/83 [==============================] - 0s 4ms/step - loss: 0.6958 - accuracy: 0.7404 - val_loss: 1.1810 - val_accuracy: 0.4000
Epoch 5/25
83/83 [==============================] - 0s 5ms/step - loss: 0.6132 - accuracy: 0.7627 - val_loss: 0.9454 - val_accuracy: 0.5273
Epoch 6/25
83/83 [==============================] - 0s 4ms/step - loss: 0.6018 - accuracy: 0.7911 - val_loss: 1.0810 - val_accuracy: 0.5091
Epoch 7/25
83/83 [==============================] - 0s 4ms/step - loss: 0.5518 - accuracy: 0.8093 - val_loss: 1.2263 - val_accuracy: 0.4545
Epoch 8/25
83/83 [==

In [9]:
Model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 13, 20)            21620     
                                                                 
 max_pooling1d (MaxPooling1D  (None, 6, 20)            0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 120)               0         
                                                                 
 dense (Dense)               (None, 512)               61952     
                                                                 
 dense_1 (Dense)             (None, 256)               131328    
                                                                 
 dense_2 (Dense)             (None, 128)               32896     
                                                        

In [18]:
Name = input('Name: ')
Model.save(Name + '.h5')
